# ANNOVAR FILES

Este programa tiene como objetivo tomar un archivo de OMIM que contiene las variantes mendelianas y regresar un archivo con extensión .avinput el cual pueda ser leido por el programa ANNOVAR. Posteriormente se obtendra la ubicación correspontiente a cada variante y podra graficarse los resultados.

### Flujo de los programas

Para generar el archivo .avinput sera necesaria la ejecución de dos códigos:

1. **getVariants** -> El cual se encarga de obtener el id de las variantes asociadas a un MIMnumber, a partir del archivo genemap2.txt proporcionado pòr OMIM.

2. **GenAvinput** -> El cual toma el archivo generado pot getVariants y genera el archivo .avinput que se le suministrara a ANNOVAR

# getVariants

### Input file

In [12]:
%%bash
head ./OMIM/genemap2.txt

# Copyright (c) 1966-2021 Johns Hopkins University. Use of this file adheres to the terms specified at https://omim.org/help/agreement.
# Generated: 2021-04-22
# See end of file for additional documentation on specific fields
# Chromosome	Genomic Position Start	Genomic Position End	Cyto Location	Computed Cyto Location	MIM Number	Gene Symbols	Gene Name	Approved Symbol	Entrez Gene ID	Ensembl Gene ID	Comments	Phenotypes	Mouse Gene Symbol/ID
chr1	0	27600000	1p36		607413	AD7CNTP	Alzheimer disease neuronal thread protein						
chr1	0	27600000	1p36		612367	ALPQTL2	Alkaline phosphatase, plasma level of, QTL 2		100196914		linkage with rs1780324	{Alkaline phosphatase, plasma level of, QTL 2}, 612367 (2)	
chr1	0	123400000	1p		606788	ANON1	Anorexia nervosa, susceptibility to, 1		171514			{Anorexia nervosa, susceptibility to, 1}, 606788 (2)	
chr1	0	27600000	1p36		605462	BCC1	Basal cell carcinoma, susceptibility to, 1		100307118		associated with rs7538876	{Basal cell carcinoma, susceptibility to, 1}

### Code

In [ ]:
import argparse

parser = argparse.ArgumentParser(description='This program creates a file with the monogenic variants in VCF format, which can be used by ANNOVAR to get their genomic location.')
parser.add_argument(
    "--input",
    metavar = "path/to/file",
    help = "Path to OMIM Gene Map File",
    required = True,
    "--varInfo",
    metavar = "path/to/file",
    help = "File path where the user will save the variants information",
    required = True,
    "--VCF",
    metavar = "path/to/file",
    help = "File path with monogenic variants in VCF format",
    required = True   
)
args = parser.parse_args()

In [ ]:
##################################################################################################################
#####################    Program for the Construction of monogenic variant files     #############################
##########################   These files are received as input by ANNOVAR   ######################################
##################################################################################################################



####################################################################################################################
####################################################################################################################
                    ############################## Functions ##############################
####################################################################################################################
####################################################################################################################



####################################################################################################################
                    ############################## OMIM ##############################
####################################################################################################################

def get_inheritance(Info,GeneType):
    '''
    **Description** -> This fucntion return the location of Mendelian diseases
    ::Info::(param): String that conteins all the monogenic variant info
    ::GeneType::(param): Dictionary that conteins all the monogenic disease inheritance keys
    '''
    
    # Search into the variant to find out if it is a Mendelian disease
    Data = None
    for key in GeneType:
        if key in Info:
            
            Info = Info.split("\t")
            
            # Get interest fields
            Chrom = Info[0]
            Start = Info[1]
            End = Info[2]
            MIMnumber = Info[5]
            
            # Return interest fields in a single tupple
            Data = (key,Chrom,Start,End,MIMnumber)
            return(Data)
    
    # If the variant isn't a Mendelian disease return Null
    return(Data)

####################################################################################################################
                    ############################## Ensembl ##############################
####################################################################################################################


def get_variant(MonogenicDisease,FilePath):
    '''
    **Description** -> This fucntion find out the variant id of a monogenic disease and then create a file that contains the info of the monogenic disease
    ::MonogenicDisease::(param): A dictionary that conteins all the Mendelian diseases locations
    ::FilePath::(param): The file path where the user will save the variant Info
    '''
    c = 0
    Header = "#ID\tChromosome\tStart\tEnd\tMIM\tInheritance\n"
    NewFile = open(FilePath,"w")
    NewFile.write(Header)
    server = "https://rest.ensembl.org"
    
    # iterate the dictionary to get the keys of the disease type
    for DiseaseType in MonogenicDisease:
        for Disease in MonogenicDisease[DiseaseType]:
            
            # Get the Mendelian disease features
            Chromosome = Disease[0].replace("chr","")
            Start = Disease[1]
            End = Disease[2]
            MIMnumber = Disease[3]
            
            # Search all the variants asociated to this chromosome location
            ext = "/phenotype/region/homo_sapiens/{}:{}-{}?feature_type=Variation;non_specified=0;trait=1;tumour=0".format(Chromosome,Start,End)
            r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
            
            decoded = r.json()
            for variant in decoded:
                try:
                    # If the variant contain a MIMnumber we have to compare if it is the MIMnumber that we was searching
                    MIM2 = variant["phenotype_associations"][0]["attributes"]["MIM"]
                    ID = variant["id"]
                    if MIMnumber == MIM2:
                        
                        # when we find the MIMnumber of interest we write in the file the variant info
                        data = (ID,Chromosome,Start,End,MIMnumber)
                        Location = variant["phenotype_associations"][0]["location"]
                        Location = Location.replace(":",",").replace("-",",")
                        Location = Location.split(",")
                        Start2 =  Location[1]
                        End2 = Location[2]
                        
                        Line = "{}\t{}\t{}\t{}\t{}\t{}\n".format(ID,Chromosome,Start2,End2,MIMnumber,DiseaseType)
                        NewFile.write(Line)
                        print(c)
                        c = c + 1
                        
                except:
                    continue
    NewFile.close()             
    return(FilePath)

####################################################################################################################
####################################################################################################################
                    ############################## Main Code ##############################
####################################################################################################################
####################################################################################################################

import requests, sys
import re

Path = "./genemap2.txt"
Monogenic = {  
            "Multifactorial":[],
            "Isolated cases":[], 
            "Digenetic recessive":[],
            "Somatic mutation":[],
            "Mitochondrial":[],
            "Inherited chromosomal imbalance":[],
            "X-linked dominant":[],
            "X-linked recessive":[],
            "Pseudoautosomal recessive":[],
            "Pseudoautosomal dominant":[],
            "Autosomal dominant":[],
            "Autosomal recessive":[],
            "X-linked":[]}


# Open genemap2 file
File = open(Path,"r")
for line in File:
    
    # Ignore commented lines
    if line.startswith("#"):
        continue
    
    # Eliminate line break and find out info by calling de function
    line = line.rstrip()
    values = get_inheritance(line,Monogenic)
    
    if values:
        Monogenic[values[0]].append(values[1:])
        
VariantPath = "./VariantFileOMIM.txt"    
get_variant(Monogenic,VariantPath)

### Output file

In [14]:
%%bash
head ../downloads/VariantFileOMIM3.txt

#ID	Chromosome	Start	End	MIM	Inheritance
rs1569231897	X	10213710	10213710	302910	X-linked dominant
rs1569233549	X	10220876	10220876	302910	X-linked dominant
rs1569226551	X	10187602	10187602	302910	X-linked dominant
rs1569230006	X	10206463	10206463	302910	X-linked dominant
rs121917889	X	11121652	11121652	300056	X-linked dominant
rs121917888	X	11120974	11120974	300056	X-linked dominant
rs387906488	X	11294802	11294810	300391	X-linked dominant
rs104894737	X	11294790	11294790	300391	X-linked dominant
rs387906491	X	11298779	11298781	300391	X-linked dominant


## GenAvinput

### Input file

In [15]:
%%bash
head ../downloads/VariantFileOMIM3.txt

#ID	Chromosome	Start	End	MIM	Inheritance
rs1569231897	X	10213710	10213710	302910	X-linked dominant
rs1569233549	X	10220876	10220876	302910	X-linked dominant
rs1569226551	X	10187602	10187602	302910	X-linked dominant
rs1569230006	X	10206463	10206463	302910	X-linked dominant
rs121917889	X	11121652	11121652	300056	X-linked dominant
rs121917888	X	11120974	11120974	300056	X-linked dominant
rs387906488	X	11294802	11294810	300391	X-linked dominant
rs104894737	X	11294790	11294790	300391	X-linked dominant
rs387906491	X	11298779	11298781	300391	X-linked dominant


### Code

In [ ]:
import urllib.request
import re

# Path to the input and output file
Path = "./VariantFileOMIM3.txt"
ANNOVAR = "./Monogenic2.avinput"

# Open files
File = open(Path,"r")
avinputFile = open(ANNOVAR,"w")
Contador = 0

for line in File:
    
    # Ignore first line of the document
    if line.startswith("#"):
        continue
    
    # Get file fields
    line = line.rstrip()
    line = line.split("\t")
    
    # Get variant Id
    Id = line[0]
    Id = Id.replace("rs","")
    
    # Get variant features
    Chromosome = line[1]
    Start = line[2]
    End = line[3]
    MIM = line[4]
    DiseaseType = line[5]

    # Get the nucleotide change of the variant from NCBI with urllib
    UrlPath = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id={}&retmode=xml'.format(Id)
    with urllib.request.urlopen(UrlPath) as response:
        html = str(response.read())
        
        # Search the nucleotide change of the variant
        SNP = re.findall('SEQ=\[.+\]', html)
        Significance = re.findall('<CLINICAL_SIGNIFICANCE>.+</CLINICAL_SIGNIFICANCE>', html)
        
        # Save the nucleotide change and the clinvar significance
        try:
            SNPVariant = SNP[0].replace("SEQ=[","").replace("]","")
            ClinvarSig = Significance[0].replace("<CLINICAL_SIGNIFICANCE>","").replace("</CLINICAL_SIGNIFICANCE>","")
            
        # If there is not nucleotide change or clinvar significance go to the next variant
        except:
            continue
        
        # Length of the nucleotide change tell us if it is a mutation, insertion or deletion
        Blength = len(SNPVariant)
        print(Contador)
        Contador = Contador + 1
        SNPVariant = SNPVariant.split("/")
        Alength = len(SNPVariant)
        
        # If Alength is greater than 2 it means that there is more than 1 polimorfism of th same variant
        if Alength > 2:
            for i in range(Alength):
                
                if i == 0:
                    continue
                
                # If the nucleotide start position is a middle dash, the variant is an insertion
                if SNPVariant[0] == "-":
                    Info = "{} {} {} {} {} comments: rs{}, Insertion, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[i],Id,DiseaseType,MIM,ClinvarSig)
                    avinputFile.write(Info)
                    continue
                
                # If the nucleotide change position is a middle dash, the variant is a deletion
                if SNPVariant[i] == "-":
                    Info = "{} {} {} {} {} comments: rs{}, Deletion, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[i],Id,DiseaseType,MIM,ClinvarSig)
                    avinputFile.write(Info)
                    continue
                
                # If the change position is greater than start position, it means that the variant is an insertion
                if len(SNPVariant[0]) < len(SNPVariant[i]):
                    Info = "{} {} {} {} {} comments: rs{}, Insertion, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[i],Id,DiseaseType,MIM,ClinvarSig)
                    avinputFile.write(Info)
                    continue
                
                # If the start position is greater than change position, it means that the variant is a deletion
                if len(SNPVariant[0]) > len(SNPVariant[i]):
                    Info = "{} {} {} {} {} comments: rs{}, Deletion, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[i],Id,DiseaseType,MIM,ClinvarSig)
                    avinputFile.write(Info)
                    continue
                    
                # If there is no difference in length after nucleotide change, the variant is a mutation  
                Info = "{} {} {} {} {} comments: rs{}, Mutation, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[i],Id,DiseaseType,MIM,ClinvarSig)
                avinputFile.write(Info)
                continue
                
        # If Alength is equal to 1 it means that there is a single polimorifism of the variant       
        else:
            
            # If the nucleotide start position is a middle dash, the variant is an insertion
            if SNPVariant[0] == "-":
                Info = "{} {} {} {} {} comments: rs{}, Insertion, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[1],Id,DiseaseType,MIM,ClinvarSig)
                avinputFile.write(Info)
                continue    
            
            # If the nucleotide change position is a middle dash, the variant is a deletion
            if SNPVariant[1] == "-":
                Info = "{} {} {} {} {} comments: rs{}, Deletion, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[1],Id,DiseaseType,MIM,ClinvarSig)
                avinputFile.write(Info)
                continue
            
            # If the change position is greater than start position, it means that the variant is an insertion
            if len(SNPVariant[0]) < len(SNPVariant[1]):
                Info = "{} {} {} {} {} comments: rs{}, Insertion, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[1],Id,DiseaseType,MIM,ClinvarSig)
                avinputFile.write(Info)
                continue
            #If the start position is greater than change position, it means that the variant is a deletion
            if len(SNPVariant[0]) > len(SNPVariant[1]):
                Info = "{} {} {} {} {} comments: rs{}, Deletion, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[1],Id,DiseaseType,MIM,ClinvarSig)
                avinputFile.write(Info)
                continue  
              
            # If there is no difference in length after nucleotide change, the variant is a mutation
            Info = "{} {} {} {} {} comments: rs{}, Mutation, {}, {}, {}\n".format(Chromosome,Start,End,SNPVariant[0],SNPVariant[1],Id,DiseaseType,MIM,ClinvarSig)
            avinputFile.write(Info)
            continue 
            
# Close files     
avinputFile.close()
File.close()

### Output file

In [3]:
%%bash
head ../../downloads/Monogenic2.avinput

X 10213710 10213710 G A comments: rs1569231897, Mutation, X-linked dominant, 302910, pathogenic
X 10220876 10220876 G C comments: rs1569233549, Mutation, X-linked dominant, 302910, pathogenic
X 10187602 10187602 G A comments: rs1569226551, Mutation, X-linked dominant, 302910, pathogenic
X 10206463 10206463 C G comments: rs1569230006, Mutation, X-linked dominant, 302910, pathogenic
X 11121652 11121652 C T comments: rs121917889, Mutation, X-linked dominant, 300056, pathogenic
X 11120974 11120974 C T comments: rs121917888, Mutation, X-linked dominant, 300056, pathogenic
X 11294802 11294810 TTTTATTTG - comments: rs387906488, Deletion, X-linked dominant, 300391, pathogenic
X 11294790 11294790 T C comments: rs104894737, Mutation, X-linked dominant, 300391, pathogenic
X 11298779 11298781 C - comments: rs387906491, Deletion, X-linked dominant, 300391, pathogenic
X 11298833 11298834 C - comments: rs387906489, Deletion, X-linked dominant, 300391, pathogenic
